## Reading 21-1 - FILE*- Reading from a File during Runtime

By the end of this reading, you will learn:
<ol>
    <li>What a file actually is</li>
    <li>FILE* I/O</li>
    <li>How to create and free File Pointers</li>
    <li>How to simply read from a FILE* using pointers</li>
</ol>

### Reading from a File during Runtime

We want to be able to develop files that read from the user. This means being able to get input in real time
<ul>
    <li><b>Benefit:</b> Being able to make portable code for users</li>
    <li><b>Drawback:</b> Your code is now at the mercy of the user</li>
</ul>

From a computing perspective, this begs the question: What exactly <i>are</i> files?
<ul>
    <li>Files are char arrays that we can access, add to, and modify</li>
    <li>Files also remain in memory after we complete the program</li>
</ul>

From our understanding of dynamic memory, we know that we need information to track the data in a File. We will use a C struct called a <code>FILE</code> to track all the information. And since files are in the dynamic memory, we can deduce that we need a <code>FILE*</code> to allocate to the dynamic memory.

### Creating a <code>FILE*</code>

In order to open a FILE*, we will use the <code>fopen</code> function from the <code>#include \<stdlib.h\></code> library

<code>FILE* fopen ( const char * filename, const char * mode );</code><br>
<ul>
    <li><code>const char * filename</code> is a string with the name of the file</li>
    <li><code>const char * mode</code> is a string that tells us what we can or cannot do with the file</li>
</ul>
    
Here is an example:
    
<code>FILE *fp = ( "filename.txt", "r" );</code><br>
<ul>
    <li>Create a pointer to the location in memory of filename.txt</li>
    <li>“r” – Read only</li>
</ul>

### <code>FILE*</code> modes

You can put one or more of these in the string for <code>mode</code> to describe the file

“r“ – read
    Open file for input operations. The file must exist.
    
"w"- write
    Create an empty file for output operations. 
    
"a"	append
    Open file for output at the end of a file. 
    
    
Here is an example:
    
<code>FILE* fp = ( "filename.txt", "rw" );</code><br>
<ul>
    <li>Create a pointer to the location in memory of filename.txt</li>
    <li>“rw” – Read and Write</li>
</ul>

### <code>FILE*</code> - Closing the file

Like any other type of dynamic memory, we need to de-allocate the pointer. The difference is that, since we do not want to delete the file, we will not free the memory.

    fclose(fp);

<b>Issue:</b> If there is no file with the filename, then the FILE pointer will be NULL
<ul>
    <li>Trying to de-allocate a NULL pointer is a segmentation fault</li>
    <li>However, we can use this to check if the file exist</li>
    <li>Rule: call fclose only in a brace procedure where one of the conditions is that the pointer is not NULL.</li>
</ul>

### Improper FILE* de-allocation - filePtrExBad.c

In this example, the file <a href = "https://github.com/mmorri22/cse20133/blob/main/readings/lec21/filePtrExBad.c">filePtrExBad.c</a> creates two <code>FILE*</code>. In this example, the file <a href ="https://raw.githubusercontent.com/mmorri22/cse20133/main/readings/lec21/filePtrExBad.c">intPtr.txt</a> exists, but the file fileDNE.txt does not exist

	/* File Name */
	char *file_fp1 = "intFile.txt";
	char *file_DNE = "fileDNE.txt";

In this code, we check to see if the file exists by comparing to <code>NULL</code>. For the correct implementation, we usit <code>exit(-1);</code> in the even the file does not exist. In the incorrect implementation, we do not exit when we determine that the file does not exist.

    // Correct implementation
	if(fp1 == NULL){
		fprintf(stderr, "The file %s does not exist\n", file_fp1);
		exit(-1);
	}
    else{
        fprintf(stout, "The file %s exist!\n", fp1);
    }
    fclose(fp1);
	
	// Incorrect implementation
	if(fpDNE == NULL){
		fprintf(stderr, "The file %s does not exist\n", file_DNE);
	}
    else{
        fprintf(stout, "The file %s exist!\n", file_DNE);
    }

	// Trying to de-allocate a NULL pointer
	fclose(fpDNE);
    
And here is an example of using <code>valgrind</code> on this program. Here are the key points:
<ul>
    <li></li>
    <li><font style="background-color:powderblue;">The output in power blue is what happens when the Makefile command is run.</font></li>
    <li><font color="white" style="background-color:green;">The output in green</font> is what you would see without valgrind.</li>
    <li><font color="white" style="background-color:red;">The output in red</font> corresponds to valgrind error messages</li>
    <ul>
        <li><code><font color="white" style="background-color:red;">==12510== Invalid read of size 4</font></code> - Means that the program tried to read from four bytes of non-existent memory</li>
        <li><code><font color="white" style="background-color:red;">==12510==    by 0x400613: main (filePtrExBad.c:49)</font></code> - Means the error occured at line 49 of <a href = "https://github.com/mmorri22/cse20133/blob/main/readings/lec21/filePtrExBad.c">filePtrExBad.c</a></li>
        <li><code><font color="white" style="background-color:red;">==12510==  Address 0x0 is not stack'd, malloc'd or (recently) free'd</font></code> - Indicates that the memory address pointer value was 0x0 - since we know that NULL pointers contain 0x0, this means we tried to read from a NULL pointer.</li>
    </ul>
    <li><code><font color="white" style="background-color:green;">make: *** [filePtrExBad] Segmentation fault (core dumped)</font></code> - Indicates we got a segmentation fault.</li>
</ul>

<b>What is all means</b>: When we tried to execute the code at line 49 of <a href = "https://github.com/mmorri22/cse20133/blob/main/readings/lec21/filePtrExBad.c">filePtrExBad.c</a>, which is:

    fclose(fpDNE);
    
What happened is that our attempt to allocate <code>FILE *fpDNE = fopen(file_DNE, "r");</code> failed because <b>file_DNE.txt did not exist</b>. So when we ran <code>fclose</code>, we could not free any pointer because the <b>FILE* wasn't pointing to any memory</b>. FILE* was actually pointing at <code>0x0</code>, which is at the beginning of the Data Heap. So we went outside of the segment, so we got a <b>segmentation fault</b>.


### <b>Here is the valgrind run:</b>

<code><font style="background-color:powderblue;">> make filePtrExBad</font></code><br>
<code><font style="background-color:powderblue;">gcc -std=c2x -O2 -g -Wall -Wextra -Wconversion -Wshadow -Werror -c filePtrExBad.c</font></code><br>
<code><font style="background-color:powderblue;">gcc -std=c2x -O2 -g -Wall -Wextra -Wconversion -Wshadow -Werror -o filePtrExBad filePtrExBad.o -lm</font></code><br>
<code><font style="background-color:powderblue;">valgrind --leak-check=full ./filePtrExBad</font></code><br>
<code>==12510== Memcheck, a memory error detector</code><br>
<code>==12510== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.</code><br>
<code>==12510== Using Valgrind-3.14.0 and LibVEX; rerun with -h for copyright info</code><br>
<code>==12510== Command: ./filePtrExBad</code><br>
<code>==12510==</code><br>
<code><font color="white" style="background-color:green;">The file intFile.txt exists!</font></code><br>
<code><font color="white" style="background-color:green;">The file fileDNE.txt does not exist</font></code><br>
<code><font color="white" style="background-color:red;">==12510== Invalid read of size 4</font></code><br>
<code>==12510==    at 0x51A7034: fclose@@GLIBC_2.2.5 (in /usr/lib64/libc-2.17.so)</code><br>
<code><font color="white" style="background-color:red;">==12510==    by 0x400613: main (filePtrExBad.c:49)</font></code><br>
<code><font color="white" style="background-color:red;">==12510==  Address 0x0 is not stack'd, malloc'd or (recently) free'd</font></code><br>
<code>==12510==</code><br>
<code>==12510==</code><br>
<code>==12510== Process terminating with default action of signal 11 (SIGSEGV)</code><br>
<code>==12510==  Access not within mapped region at address 0x0</code><br>
<code>==12510==    at 0x51A7034: fclose@@GLIBC_2.2.5 (in /usr/lib64/libc-2.17.so)</code><br>
<code>==12510==    by 0x400613: main (filePtrExBad.c:49)</code><br>
<code>==12510==  If you believe this happened as a result of a stack</code><br>
<code>==12510==  overflow in your program's main thread (unlikely but</code><br>
<code>==12510==  possible), you can try to increase the size of the</code><br>
<code>==12510==  main thread stack using the --main-stacksize= flag.</code><br>
<code>==12510==  The main thread stack size used in this run was 8388608.</code><br>
<code>==12510==</code><br>
<code><font style="background-color:cyan;">==12510== HEAP SUMMARY:</font></code><br>
<code><font style="background-color:cyan;">==12510==     in use at exit: 0 bytes in 0 blocks</font></code><br>
<code><font style="background-color:cyan;">==12510==   total heap usage: 2 allocs, 2 frees, 1,136 bytes allocated</font></code><br>
<code><font style="background-color:cyan;">==12510==</font></code><br>
<code><font style="background-color:cyan;">==12510== All heap blocks were freed -- no leaks are possible</font></code><br>
<code>==12510==</code><br>
<code>==12510== For counts of detected and suppressed errors, rerun with: -v</code><br>
<code><font color="white" style="background-color:red;">==12510== ERROR SUMMARY: 1 errors from 1 contexts (suppressed: 0 from 0)</font></code><br>
<code><font color="white" style="background-color:green;">make: *** [filePtrExBad] Segmentation fault (core dumped)</font></code><br>
<code><font color="white" style="background-color:green;">make: *** Deleting file `filePtrExBad'</font></code><br>

### The Fixed Version

The code <a href = "https://github.com/mmorri22/cse20133/blob/main/readings/lec21/filePtrExBad.c">filePtrExGood.c</a> shows the correct implementation. 

The three-step method for allocating FILE* are
<ol>
    <li>Allocate the FILE* using the file name and the file mode</li>
    <li>Check to ensure that the file exists by comparing FILE* to NULL. If it does not exist, run <code>exit(-1)</code></li>
    <li>When you are done with the file, close using fclose</li>
</ol>

	char *file_fp1 = "intFile.txt";
	char *file_DNE = "fileDNE.txt";

	/* Allocate File Pointers */
	FILE *fp1 = fopen(file_fp1, "r");
	FILE *fpDNE = fopen(file_DNE, "r");

	// Correct implementation
	if(fp1 == NULL){
		fprintf(stderr, "The file %s does not exist\n", file_fp1);
		exit(-1);
	}
	else{
			fprintf(stdout, "The file %s exists!\n", file_fp1);
	}
	fclose(fp1);


	if(fpDNE == NULL){
		fprintf(stderr, "The file %s does not exist\n", file_DNE);
		exit(-1);
	}
	else{
			fprintf(stdout, "The file %s exists!\n", file_DNE);
	}
	fclose(fpDNE);

### <font color = "red">Class Introduction Question #1 - What is a <code>FILE*</code> and why are they used to keep track of files?</a>

### <font color = "red">Class Introduction Question #2 - Describe how to create a FILE* struct, and the three types of FILE* modes</a>

### <font color = "red">Class Introduction Question #3 - Describe the three-step process for allocating and freeing a FILE* and why it is important to follow these three steps?</a>

### The final reading for this lecture is <a href = "https://github.com/mmorri22/cse20133/blob/main/readings/lec21/Reading%2021-2.ipynb">Reading 21-2 - Combining Concepts - stdout and FILE*</a>